# 사용되는 데이터
* event_bq.csv
* items_bq.csv

# 1. Bigquery에서 데이터 추출하기

 - Google Cloud의 BigQuery에서 진행

## 1-1. event_bq.csv 생성하기

**event_bq.csv 컬럼 구성**
  * user_id
  * ga_session_id
  * source
  * medium
  * campaign
  * country
  * region
  * city
  * device
  * type
  * item_id
  * event_date

**작성된 sql 쿼리**



```sql
SELECT
  --  유저 ID (user_pseudo_id를 통해 생성)
  user_pseudo_id AS user_id,

  --  세션 ID
  (SELECT value.int_value
   FROM UNNEST(event_params)
   WHERE key = "ga_session_id") AS ga_session_id,

  --  트래픽 유입경로 정보
  traffic_source.source AS source,       
  traffic_source.medium AS medium,      
  traffic_source.name AS campaign,       

  --  지역 정보
  geo.country AS country,
  geo.region AS region,
  geo.city AS city,

  --  기기 정보
  device.category AS device,

  --  이벤트명
  event_name AS type,

  --  상품 정보
  item.item_id AS item_id,

  --  이벤트 발생 시간
  FORMAT_TIMESTAMP(
    '%Y-%m-%d %H:%M:%S',
    TIMESTAMP_MICROS(event_timestamp)
  ) AS event_date

FROM
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
LEFT JOIN
  UNNEST(items) AS item

WHERE
  _TABLE_SUFFIX BETWEEN '20201201' AND '20210201'
  AND event_timestamp IS NOT NULL

ORDER BY
  user_id,
  event_date;


```



In [ ]:
# event_bq.csv 불러오기
import pandas as pd
event_bq = pd.read_csv("/content/event_bq.csv")

In [ ]:
# 데이터 확인하기
event_bq

,사용자ID,세션_식별자,유입출처,유입유형,캠페인명,국가,지역,도시,기기,타입,상품ID,이벤트발생시간
0,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:09:36
1,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:09:36
2,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,scroll,NaN,2021-01-20 08:10:30
3,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:10:36
4,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:10:36
...,...,...,...,...,...,...,...,...,...,...,...,...
5325904,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEGBGJ132699,2020-12-05 09:00:44
5325905,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEGABB137799,2020-12-05 09:00:44
5325906,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEACBA129599,2020-12-05 09:00:44
5325907,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEAFBA115399,2020-12-05 09:00:44


## 1-2. items_bq.csv 생성하기

**items_bq.csv 컬럼 구성**
  * id
  * name



```sql
SELECT
  item.item_id AS id,
  item.item_name AS name
FROM
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`,
  UNNEST(items) AS item
WHERE
  _TABLE_SUFFIX BETWEEN '20201101' AND '20210201'
GROUP BY
  id, name
ORDER BY
  name;

```



In [ ]:
# items_bq.csv 불러오기
items_bq = pd.read_csv("/content/items_bq.csv")

In [ ]:
# 데이터 확인하기
items_bq

,상품ID,상품명
0,9196902,#IamRemarkable Journal
1,GGCOGOCC101199,#IamRemarkable Journal
2,9196914,#IamRemarkable Ladies T-Shirt
3,GGCOGXXX1007,#IamRemarkable Ladies T-Shirt
4,9196913,#IamRemarkable Ladies T-Shirt
...,...,...
1396,9184961,YouTube Women's Favorite Tee White
1397,GGOEGXXX0919,YouTube Women's Favorite Tee White
1398,9195907,Youth Jumbo Print Tee White
1399,9195908,Youth Jumbo Print Tee White


# 2. 데이터셋 컬럼명 한글화

## 2-1. event_bq 컬럼명 한글화

In [ ]:
# 컬럼명 변경하기
event_bq.columns = [
    "사용자ID",
    "세션_식별자",
    "유입출처",
    "유입유형",
    "캠페인명",
    "국가",
    "지역",
    "도시",
    "기기",
    "타입",
    "상품ID",
    "이벤트발생시간"
]

# 확인
event_bq.head()

,사용자ID,세션_식별자,유입출처,유입유형,캠페인명,국가,지역,도시,기기,타입,상품ID,이벤트발생시간
0,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:09:36
1,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:09:36
2,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,scroll,NaN,2021-01-20 08:10:30
3,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:10:36
4,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:10:36


In [ ]:
# 파일 다시 저장하기
event_bq.to_csv("/content/event_bq.csv", index=False, encoding="utf-8-sig")


In [ ]:
event_bq

,사용자ID,세션_식별자,유입출처,유입유형,캠페인명,국가,지역,도시,기기,타입,상품ID,이벤트발생시간
0,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:09:36
1,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:09:36
2,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,scroll,NaN,2021-01-20 08:10:30
3,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:10:36
4,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:10:36
...,...,...,...,...,...,...,...,...,...,...,...,...
5325904,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEGBGJ132699,2020-12-05 09:00:44
5325905,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEGABB137799,2020-12-05 09:00:44
5325906,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEACBA129599,2020-12-05 09:00:44
5325907,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEAFBA115399,2020-12-05 09:00:44


## 2-2. items_bq 컬럼명 한글화

In [ ]:
# 컬럼명 변경하기
items_bq.columns = [
    "상품ID",
    "상품명"
]

# 확인
items_bq.head()

,상품ID,상품명
0,9196902,#IamRemarkable Journal
1,GGCOGOCC101199,#IamRemarkable Journal
2,9196914,#IamRemarkable Ladies T-Shirt
3,GGCOGXXX1007,#IamRemarkable Ladies T-Shirt
4,9196913,#IamRemarkable Ladies T-Shirt


In [ ]:
# 파일 다시 저장하기
items_bq.to_csv("/content/items_bq.csv", index=False, encoding="utf-8-sig")

In [ ]:
items_bq

,상품ID,상품명
0,9196902,#IamRemarkable Journal
1,GGCOGOCC101199,#IamRemarkable Journal
2,9196914,#IamRemarkable Ladies T-Shirt
3,GGCOGXXX1007,#IamRemarkable Ladies T-Shirt
4,9196913,#IamRemarkable Ladies T-Shirt
...,...,...
1396,9184961,YouTube Women's Favorite Tee White
1397,GGOEGXXX0919,YouTube Women's Favorite Tee White
1398,9195907,Youth Jumbo Print Tee White
1399,9195908,Youth Jumbo Print Tee White


# 3. 데이터셋 병합하기

## 3-1. event_bq 데이터, items_bq 데이터 병합

In [ ]:
# '상품ID'기준 조인
bq = pd.merge(event_bq, items_bq, how="left", left_on="상품ID", right_on="상품ID")
bq

,사용자ID,세션_식별자,유입출처,유입유형,캠페인명,국가,지역,도시,기기,타입,상품ID,이벤트발생시간,상품명
0,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:09:36,NaN
1,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:09:36,NaN
2,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,scroll,NaN,2021-01-20 08:10:30,NaN
3,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:10:36,NaN
4,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:10:36,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5357472,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEGBGJ132699,2020-12-05 09:00:44,Google Tech Taco
5357473,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEGABB137799,2020-12-05 09:00:44,Google LoveHandle Black
5357474,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEACBA129599,2020-12-05 09:00:44,Android Iconic Pin
5357475,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEAFBA115399,2020-12-05 09:00:44,Snowflake Android Cardboard Sculpture


## 3-2. 통합 데이터셋 최종 저장

In [ ]:
# 파일 저장하기
bq.to_csv("/content/통합데이터셋_bq(dataton).csv", index=False, encoding="utf-8-sig")


# 4. 최종 데이터셋 확인

In [ ]:
result = pd.read_csv("/content/통합데이터셋_bq(dataton).csv")

In [ ]:
result

,사용자ID,세션_식별자,유입출처,유입유형,캠페인명,국가,지역,도시,기기,타입,상품ID,이벤트발생시간,상품명
0,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:09:36,NaN
1,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:09:36,NaN
2,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,scroll,NaN,2021-01-20 08:10:30,NaN
3,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,add_payment_info,NaN,2021-01-20 08:10:36,NaN
4,8.014384e+06,7289119443,(direct),(none),(direct),India,Uttar Pradesh,Noida,mobile,page_view,NaN,2021-01-20 08:10:36,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5357472,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEGBGJ132699,2020-12-05 09:00:44,Google Tech Taco
5357473,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEGABB137799,2020-12-05 09:00:44,Google LoveHandle Black
5357474,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEACBA129599,2020-12-05 09:00:44,Android Iconic Pin
5357475,5.075019e+07,628401593,<Other>,<Other>,<Other>,Canada,Quebec,Gatineau,desktop,view_item,GGOEAFBA115399,2020-12-05 09:00:44,Snowflake Android Cardboard Sculpture
